# Tableau Project - Videogame Co.

## Where do we open our new office?

### Instrucciones
#### Somos una compañía de Videogames de 50 empleados. Tenemos que abrir una oficina nueva, pero, ¿dónde?
#### Contamos con 20 desarrolladores, 20 UX/UI y 10 ejecutivos.
#### Tiene que haber alrededor:
#### 1-. Compañías de Software
#### 2-. Buen ratio entre grades compañías y start-ups
#### 3-. Empresas de diseño, diseño web, relacionado con UX
#### 4-. Restaurantes para que los ejecutivos puedan reunirse y hacer negocios 
#### 5-. Evitar compañías muy viejas alrededor

### Variables interesantes que mirar
#### Total money raised (inversiones), competitors, latitud y longitud de oficinas, twitter (influencia de la empresa, API tweety), cathegory code

In [58]:
import re
import pandas as pd
import pymongo
MongoClient = pymongo.MongoClient
client = MongoClient()
db = client.companies

In [ ]:
def toGeoJSON(array):
    return {
        "type":"Point",
        "coordinates":[array[0],array[1]]
    }

def convertOffice(companies):
    return {
        "name": companies["name"],
        "cuisine": companies["cuisine"],
        "position": toGeoJSON(companies["address"]["coord"])
    }

cursor = db.restaurants.find({"address.coord":{"$size":2}},{"_id":0})
geoRestaurants = list(map(lambda r: convertRestaurant(r),cursor))

In [ ]:
{offices: {$size: 0},category_code: 'web'}

In [59]:
# Para evitar compañías viejas alrededor, vamos a considerar qué es una compañía vieja, respondiendo a: ¿En qué año se fundaron
# las compañías más importantes de tecnología hoy en día? Google en el 1998, Intel 1968, Microsoft 1975, Apple 1976
# Tenemos en cuenta las compañías más importantes, ordenadas por valuation amount descending.

#Primero preguntamos a mongo por 1000 compañías de Software para sacar su lat y long.
#Software Co

cursor1 = db.companies.find({"$and": [{"category_code": 'software'}, {"offices.latitude": {"$not": re.compile("^N.*")}}, {"founded_year": {"$gte": 1965}} ]}).sort("ipo.valuation_amount", pymongo.DESCENDING).limit(1000)

software_co = list(map(lambda x: [x["name"], x["offices"]], list(cursor1)))

software = pd.DataFrame(software_co)

In [60]:
#También preguntamos a mongo por 500 compañías de Mobile para sacar su lat y long, nos interesa tener empresas de movil cerca para sacar videojuegos en el movil.
#Mobile Co

cursor2 = db.companies.find({"$and": [{"category_code": 'mobile'}, {"offices.latitude": {"$not": re.compile("^N.*")}}, {"founded_year": {"$gte": 1965}} ]}).sort("ipo.valuation_amount", pymongo.DESCENDING).limit(500)

mobile_co = list(map(lambda x: [x["name"], x["offices"]], list(cursor2)))

mobile = pd.DataFrame(mobile_co)

In [61]:
#También nos interesa saber dónde está la competencia, ya que significa que hay mercado en esa zona. 
#Preguntamos a mongo por 1000 compañías de VideoGames para sacar su lat y long.
#VideoGames Co

cursor3 = db.companies.find({"$and": [{"category_code": 'games_video'}, {"offices.latitude": {"$not": re.compile("^N.*")}}, {"founded_year": {"$gte": 1965}} ]}).sort("ipo.valuation_amount", pymongo.DESCENDING).limit(1000)

gamesvideo_co = list(map(lambda x: [x["name"], x["offices"]], list(cursor3)))

gamesvideo = pd.DataFrame(gamesvideo_co)


In [62]:
cos = software + mobile + gamesvideo
cos.head()

,0,1
0,BMC SoftwareMixiGREE,"[{'description': '', 'address1': '2101 City We..."
1,RackspaceTelenavTudou,"[{'description': 'HQ', 'address1': '5000 Walze..."
2,Jive SoftwareMeru NetworksDeNA,"[{'description': 'Palo Alto HQ', 'address1': '..."
3,QlikTechHelioDeNA,"[{'description': 'World Headquarters', 'addres..."
4,Cornerstone OnDemandJingle NetworksJoost,"[{'description': 'Headquarters', 'address1': '..."
